# 코로나 분석 프로젝트

Johns Hopkins University 에서 제공하는 코로나 데이터들을 받아온다.

In [3]:
import os 
import requests
for filename in ['time_series_covid19_confirmed_global.csv',
                 'time_series_covid19_deaths_global.csv',
                 'time_series_covid19_recovered_global.csv',
                 'time_series_covid19_confirmed_US.csv',
                 'time_series_covid19_deaths_US.csv']: 
    print(f'Downloading {filename}') 
    url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/{filename}' 
    myfile = requests.get(url) 
    open(filename, 'wb').write(myfile.content)




컬럼명을 1/22/20 -> 2020-01-20 이런 날짜형식으로 쉽게 변경을 한다.

In [4]:
from datetime import datetime 
import pandas as pd 
def _convert_date_str(df): 
    try: 
        df.columns = list(df.columns[:4]) + [datetime.strptime(d, "%m/%d/%y").date().strftime("%Y-%m-%d") for d in df.columns[4:]] 
    except: 
        print('_convert_date_str failed with %y, try %Y') 
        df.columns = list(df.columns[:4]) + [datetime.strptime(d, "%m/%d/%Y").date().strftime("%Y-%m-%d") for d in df.columns[4:]] 
        
confirmed_global_df = pd.read_csv('time_series_covid19_confirmed_global.csv') 
_convert_date_str(confirmed_global_df) 
deaths_global_df = pd.read_csv('time_series_covid19_deaths_global.csv') 
_convert_date_str(deaths_global_df) 
recovered_global_df = pd.read_csv('time_series_covid19_recovered_global.csv') 
_convert_date_str(recovered_global_df)

In [5]:
deaths_global_df

,Province/State,Country/Region,Lat,Long,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2021-10-27,2021-10-28,2021-10-29,2021-10-30,2021-10-31,2021-11-01,2021-11-02,2021-11-03,2021-11-04,2021-11-05
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,7266,7268,7269,7272,7280,7281,7281,7284,7284,7284
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,2893,2902,2909,2916,2924,2931,2937,2940,2944,2948
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,5904,5907,5913,5918,5920,5924,5927,5931,5936,5939
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,130,130,130,130,130,130,130,130,130,130
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,1703,1705,1707,1708,1710,1713,1713,1716,1718,1719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,NaN,Vietnam,14.058324,108.277199,0,0,0,0,0,0,...,21856,21910,21966,22030,22083,22131,22205,22283,22342,22412
276,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,4651,4657,4663,4663,4663,4681,4689,4694,4697,4705
277,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,1868,1874,1877,1880,1889,1890,1894,1897,1901,1905
278,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,3660,3660,3660,3661,3661,3661,3661,3661,3662,3662


## 데이터 클랜징 작업

데이터 클랜징 작업 입니다. 아래 코드는 일부 음수가 있는 값들과 cruise ship의 감염자 수치가 잘못 들어간 것들을 정제한다.

In [6]:
import numpy as np 

removed_states = "Recovered|Grand Princess|Diamond Princess" 
removed_countries = "US|The West Bank and Gaza" 

confirmed_global_df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)
deaths_global_df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True) 
recovered_global_df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)

confirmed_global_df = confirmed_global_df[~confirmed_global_df["Province_State"].replace(np.nan, "nan").str.match(removed_states)]
deaths_global_df = deaths_global_df[~deaths_global_df["Province_State"].replace(np.nan, "nan").str.match(removed_states)] 
recovered_global_df = recovered_global_df[~recovered_global_df["Province_State"].replace(np.nan, "nan").str.match(removed_states)] 

confirmed_global_df = confirmed_global_df[~confirmed_global_df["Country_Region"].replace(np.nan, "nan").str.match(removed_countries)] 
deaths_global_df = deaths_global_df[~deaths_global_df["Country_Region"].replace(np.nan, "nan").str.match(removed_countries)]
recovered_global_df = recovered_global_df[~recovered_global_df["Country_Region"].replace(np.nan, "nan").str.match(removed_countries)]




confirmed_global_df, deaths_global_df, deaths_global_df 3개의 df를 각각 melt 함수를 사용하여<br>
confirmed_global_melt_df, deaths_global_melt_df, recovered_global_melt_df 각각 다른 df로 만들어준다.

In [7]:
confirmed_global_melt_df = confirmed_global_df.melt( 
    id_vars=['Country_Region', 'Province_State', 'Lat', 'Long'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='ConfirmedCases') 
deaths_global_melt_df = deaths_global_df.melt( 
    id_vars=['Country_Region', 'Province_State', 'Lat', 'Long'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='Deaths')
recovered_global_melt_df = deaths_global_df.melt( 
    id_vars=['Country_Region', 'Province_State', 'Lat', 'Long'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='Recovered')




confirmed_global_df를 기준으로 기존 279개의 entries가 181157로 늘어난 것을 확인 할 수 있습니다.

In [8]:
confirmed_global_df.info()
print()
confirmed_global_melt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277 entries, 0 to 279
Columns: 658 entries, Province_State to 2021-11-05
dtypes: float64(2), int64(654), object(2)
memory usage: 1.4+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181158 entries, 0 to 181157
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Country_Region  181158 non-null  object 
 1   Province_State  55590 non-null   object 
 2   Lat             179850 non-null  float64
 3   Long            179850 non-null  float64
 4   Date            181158 non-null  object 
 5   ConfirmedCases  181158 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 8.3+ MB


train이라는 이름으로 confirmed_global_melt_df, deaths_global_melt_df, recovered_global_melt_df 각각으로 흩어져 있던 데이터를 하나로 합치는 과정을 진행 합니다.


In [9]:
train = confirmed_global_melt_df.merge(deaths_global_melt_df, on=['Country_Region', 'Province_State', 'Lat', 'Long', 'Date']) 
train = train.merge(recovered_global_melt_df, on=['Country_Region', 'Province_State', 'Lat', 'Long', 'Date'])
train




,Country_Region,Province_State,Lat,Long,Date,ConfirmedCases,Deaths,Recovered
0,Afghanistan,NaN,33.939110,67.709953,2020-01-22,0,0,0
1,Albania,NaN,41.153300,20.168300,2020-01-22,0,0,0
2,Algeria,NaN,28.033900,1.659600,2020-01-22,0,0,0
3,Andorra,NaN,42.506300,1.521800,2020-01-22,0,0,0
4,Angola,NaN,-11.202700,17.873900,2020-01-22,0,0,0
...,...,...,...,...,...,...,...,...
181153,Vietnam,NaN,14.058324,108.277199,2021-11-05,953547,22412,22412
181154,West Bank and Gaza,NaN,31.952200,35.233200,2021-11-05,455099,4705,4705
181155,Yemen,NaN,15.552727,48.516388,2021-11-05,9843,1905,1905
181156,Zambia,NaN,-13.133897,27.849332,2021-11-05,209852,3662,3662


# US데이터

날짜 컬럼명을 바꾸고, 필요없는 컬럼들을 drop해준다.

In [10]:
confirmed_us_df = pd.read_csv('time_series_covid19_confirmed_US.csv') 
deaths_us_df = pd.read_csv('time_series_covid19_deaths_US.csv')

confirmed_us_df.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key'], inplace=True, axis=1) 
deaths_us_df.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key', 'Population'], inplace=True, axis=1) 

confirmed_us_df.rename({'Long_': 'Long'}, axis=1, inplace=True) 
deaths_us_df.rename({'Long_': 'Long'}, axis=1, inplace=True)

_convert_date_str(confirmed_us_df) 
_convert_date_str(deaths_us_df)



데이터 클랜징 과정

In [11]:
confirmed_us_df = confirmed_us_df[~confirmed_us_df.Province_State.str.match("Diamond Princess|Grand Princess|Recovered|Northern Mariana Islands|American Samoa")] 
deaths_us_df = deaths_us_df[~deaths_us_df.Province_State.str.match("Diamond Princess|Grand Princess|Recovered|Northern Mariana Islands|American Samoa")]




일부 데이터 합치기

In [12]:
confirmed_us_df = confirmed_us_df.groupby(['Country_Region', 'Province_State']).sum().reset_index()
deaths_us_df = deaths_us_df.groupby(['Country_Region', 'Province_State']).sum().reset_index()



위도 경도 데이터 drop

In [13]:
confirmed_us_df.drop(['Lat', 'Long'], inplace=True, axis=1)
deaths_us_df.drop(['Lat', 'Long'], inplace=True, axis=1)



melt함수로 unpivot 해준다. <br>
train_us를 새롭게 생성한다.

In [14]:
confirmed_us_melt_df = confirmed_us_df.melt( 
    id_vars=['Country_Region', 'Province_State'], value_vars=confirmed_us_df.columns[2:], var_name='Date'
    , value_name='ConfirmedCases') 
deaths_us_melt_df = deaths_us_df.melt( 
    id_vars=['Country_Region', 'Province_State'], value_vars=deaths_us_df.columns[2:], var_name='Date'
    , value_name='Deaths')
train_us = confirmed_us_melt_df.merge(deaths_us_melt_df, on=['Country_Region', 'Province_State', 'Date'])



데이터 정리가 잘됌

In [15]:
train_us

,Country_Region,Province_State,Date,ConfirmedCases,Deaths
0,US,Alabama,2020-01-22,0,0
1,US,Alaska,2020-01-22,0,0
2,US,Arizona,2020-01-22,0,0
3,US,Arkansas,2020-01-22,0,0
4,US,California,2020-01-22,0,0
...,...,...,...,...,...
35311,US,Virginia,2021-11-05,933542,14125
35312,US,Washington,2021-11-05,737698,8798
35313,US,West Virginia,2021-11-05,276267,4518
35314,US,Wisconsin,2021-11-05,899647,9542


In [16]:
import pandas as pd

global 데이터 train과 train_us 데이터를 합친다.

In [17]:

train = pd.concat([train, train_us], axis=0, sort=False) 
train_us.rename({'Country_Region': 'country', 'Province_State': 'province', 'Date': 'date', 'ConfirmedCases': 'confirmed', 'Deaths': 'fatalities'}, axis=1, inplace=True)
train_us['country_province'] = train_us['country'].fillna('') + '/' + train_us['province'].fillna('')




In [18]:
train

,Country_Region,Province_State,Lat,Long,Date,ConfirmedCases,Deaths,Recovered
0,Afghanistan,NaN,33.93911,67.709953,2020-01-22,0,0,0.0
1,Albania,NaN,41.15330,20.168300,2020-01-22,0,0,0.0
2,Algeria,NaN,28.03390,1.659600,2020-01-22,0,0,0.0
3,Andorra,NaN,42.50630,1.521800,2020-01-22,0,0,0.0
4,Angola,NaN,-11.20270,17.873900,2020-01-22,0,0,0.0
...,...,...,...,...,...,...,...,...
35311,US,Virginia,NaN,NaN,2021-11-05,933542,14125,NaN
35312,US,Washington,NaN,NaN,2021-11-05,737698,8798,NaN
35313,US,West Virginia,NaN,NaN,2021-11-05,276267,4518,NaN
35314,US,Wisconsin,NaN,NaN,2021-11-05,899647,9542,NaN


합쳐진 train 데이터 셋의 컬럼명을 일부 변경 하도록 하겠습니다. 그리고, country_privince 라는 컬럼을 추가 하여 <br>country와 province 값을 구분하여 넣는다



In [19]:
train.rename({'Country_Region': 'country', 'Province_State': 'province', 'Id': 'id', 'Date': 'date', 'ConfirmedCases': 'confirmed', 'Deaths': 'fatalities', 'Recovered': 'recovered'}, axis=1, inplace=True) 
train['country_province'] = train['country'].fillna('') + '/' + train['province'].fillna('')




In [20]:
train

,country,province,Lat,Long,date,confirmed,fatalities,recovered,country_province
0,Afghanistan,NaN,33.93911,67.709953,2020-01-22,0,0,0.0,Afghanistan/
1,Albania,NaN,41.15330,20.168300,2020-01-22,0,0,0.0,Albania/
2,Algeria,NaN,28.03390,1.659600,2020-01-22,0,0,0.0,Algeria/
3,Andorra,NaN,42.50630,1.521800,2020-01-22,0,0,0.0,Andorra/
4,Angola,NaN,-11.20270,17.873900,2020-01-22,0,0,0.0,Angola/
...,...,...,...,...,...,...,...,...,...
35311,US,Virginia,NaN,NaN,2021-11-05,933542,14125,NaN,US/Virginia
35312,US,Washington,NaN,NaN,2021-11-05,737698,8798,NaN,US/Washington
35313,US,West Virginia,NaN,NaN,2021-11-05,276267,4518,NaN,US/West Virginia
35314,US,Wisconsin,NaN,NaN,2021-11-05,899647,9542,NaN,US/Wisconsin


ww_df라는 데이터 프레임을 만들어서 그안에 전세계 데이터의 new_case와 growth factor 값을 날짜별로 구한다.

In [21]:
ww_df = train.groupby('date')[['confirmed', 'fatalities']].sum().reset_index() 
ww_df['new_case'] = ww_df['confirmed'] - ww_df['confirmed'].shift(1)
ww_df['growth_factor'] = ww_df['new_case'] / ww_df['new_case'].shift(1)
ww_df.tail()



,date,confirmed,fatalities,new_case,growth_factor
649,2021-11-01,247144213,5006348,431651.0,1.408640
650,2021-11-02,247574422,5013603,430209.0,0.996659
651,2021-11-03,248091949,5021896,517527.0,1.202966
652,2021-11-04,248617813,5029528,525864.0,1.016109
653,2021-11-05,249133970,5038365,516157.0,0.981541


In [22]:
ww_melt_df = pd.melt(ww_df, id_vars=['date'], value_vars=['confirmed', 'fatalities', 'new_case']) 
ww_melt_df



,date,variable,value
0,2020-01-22,confirmed,557.0
1,2020-01-23,confirmed,655.0
2,2020-01-24,confirmed,941.0
3,2020-01-25,confirmed,1434.0
4,2020-01-26,confirmed,2118.0
...,...,...,...
1957,2021-11-01,new_case,431651.0
1958,2021-11-02,new_case,430209.0
1959,2021-11-03,new_case,517527.0
1960,2021-11-04,new_case,525864.0
